In [1]:
!pip install -q tensorflow_recommenders
!pip install -q ScaNN

     |████████████████████████████████| 85 kB 934 kB/s 
     |████████████████████████████████| 10.9 MB 2.7 MB/s 


In [87]:
from google.colab import files
files.upload()

Saving rec_data.csv to rec_data.csv


{'rec_data.csv': b'User_Id,Class_Id,Order_Rating\r\n228,1,4\r\n263,28,4\r\n310,4,4\r\n72,27,1\r\n238,25,5\r\n183,28,5\r\n269,39,4\r\n271,28,2\r\n62,41,4\r\n346,45,1\r\n352,8,5\r\n250,38,3\r\n100,42,2\r\n234,36,3\r\n185,29,2\r\n21,2,1\r\n284,21,1\r\n71,47,3\r\n239,32,1\r\n6,48,2\r\n86,2,1\r\n117,12,2\r\n50,4,2\r\n198,14,5\r\n215,46,1\r\n186,32,2\r\n47,8,2\r\n113,42,1\r\n25,16,4\r\n317,9,1\r\n70,2,3\r\n323,10,1\r\n78,39,4\r\n198,23,4\r\n142,5,4\r\n148,19,3\r\n351,8,1\r\n243,29,3\r\n150,2,5\r\n27,17,3\r\n229,13,2\r\n81,19,5\r\n257,40,5\r\n127,30,5\r\n284,49,4\r\n342,45,5\r\n220,28,2\r\n330,38,3\r\n136,37,2\r\n112,26,4\r\n163,28,2\r\n222,40,5\r\n238,37,2\r\n88,44,5\r\n239,10,1\r\n233,7,4\r\n177,10,5\r\n165,11,4\r\n38,21,4\r\n312,40,4\r\n74,43,5\r\n142,13,3\r\n231,38,4\r\n263,34,3\r\n46,7,1\r\n6,35,5\r\n90,45,4\r\n78,16,3\r\n62,15,2\r\n99,36,5\r\n324,5,1\r\n326,38,5\r\n106,35,3\r\n301,32,5\r\n268,2,2\r\n25,38,1\r\n209,25,4\r\n81,46,5\r\n344,19,4\r\n286,21,3\r\n5,13,5\r\n124,13,5\r\n241,31,2

In [3]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs

import pandas as pd

In [88]:
dataframe = pd.read_csv("/content/rec_data.csv")
dataframe[["User_Id", "Class_Id"]] = dataframe[["User_Id", "Class_Id"]].astype(str)
dataframe[["Order_Rating"]] = dataframe[["Order_Rating"]].astype(float)
dataframe

,User_Id,Class_Id,Order_Rating
0,228,1,4.0
1,263,28,4.0
2,310,4,4.0
3,72,27,1.0
4,238,25,5.0
...,...,...,...
995,34,24,4.0
996,143,22,3.0
997,122,3,4.0
998,160,16,3.0


In [89]:
inter_dict = dataframe.groupby(["User_Id", "Class_Id"])["Order_Rating"].agg(np.mean).reset_index()
print(inter_dict)
inter_dict = {name: np.array(value) for name, value in inter_dict.items()}
interaction = tf.data.Dataset.from_tensor_slices(inter_dict)

prod_dict = dataframe[["Class_Id"]].drop_duplicates()
prod_dict = {name: np.array(value) for name, value in prod_dict.items()}
product = tf.data.Dataset.from_tensor_slices(prod_dict)

interaction = interaction.map(lambda x: {
    "User_Id": x["User_Id"],
    "Class_Id": x["Class_Id"],
    "Order_Rating": float(x["Order_Rating"])
})
product = product.map(lambda x: x["Class_Id"])

    User_Id Class_Id  Order_Rating
0         1       14           1.0
1        10       19           4.0
2        10       37           1.0
3       100       33           5.0
4       100       42           2.0
..      ...      ...           ...
959      97       14           3.0
960      98       33           5.0
961      98       37           5.0
962      99       36           5.0
963      99       43           4.0

[964 rows x 3 columns]


In [90]:
user_ids = interaction.batch(100).map(lambda x: x["User_Id"])
product_ids = interaction.batch(100).map(lambda x: x["Class_Id"])

unique_user = np.unique(np.concatenate(list(user_ids)))
unique_product = np.unique(np.concatenate(list(product_ids)))
print(unique_user)
print(unique_product)

[b'1' b'10' b'100' b'101' b'102' b'103' b'104' b'105' b'106' b'107' b'108'
 b'109' b'11' b'110' b'111' b'112' b'113' b'114' b'115' b'116' b'117'
 b'118' b'119' b'12' b'120' b'121' b'122' b'123' b'124' b'125' b'126'
 b'127' b'128' b'129' b'13' b'130' b'131' b'132' b'134' b'135' b'136'
 b'137' b'139' b'14' b'140' b'141' b'142' b'143' b'144' b'145' b'146'
 b'147' b'148' b'149' b'15' b'150' b'151' b'152' b'153' b'154' b'155'
 b'156' b'157' b'158' b'159' b'16' b'160' b'161' b'162' b'163' b'164'
 b'165' b'166' b'167' b'168' b'169' b'17' b'170' b'171' b'172' b'173'
 b'174' b'175' b'176' b'177' b'178' b'179' b'18' b'180' b'182' b'183'
 b'185' b'186' b'187' b'188' b'189' b'19' b'190' b'191' b'192' b'193'
 b'194' b'195' b'196' b'197' b'198' b'199' b'2' b'20' b'200' b'201' b'202'
 b'203' b'204' b'205' b'206' b'207' b'208' b'209' b'21' b'210' b'211'
 b'212' b'213' b'214' b'215' b'216' b'217' b'218' b'219' b'22' b'220'
 b'221' b'222' b'223' b'224' b'225' b'226' b'227' b'228' b'229' b'23'
 b'230' b'

In [99]:
tf.random.set_seed(42)
shuffled = interaction.shuffle(100, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(800)
test = shuffled.skip(800).take(164)

cached_train = train.shuffle(100).batch(128)
cached_test = test.batch(256).cache()

In [100]:
class Model(tfrs.models.Model):
  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    super().__init__()

    embed_dim = 32

    self.product_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.StringLookup(vocabulary = unique_product, mask_token = None),
          tf.keras.layers.Embedding(len(unique_product) + 1, embed_dim)
    ])

    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.StringLookup(vocabulary = unique_user, mask_token = None),
          tf.keras.layers.Embedding(len(unique_user) + 1, embed_dim)
    ])

    self.rating_model = tf.keras.Sequential([
          tf.keras.layers.Dense(256, activation="relu"),
          tf.keras.layers.Dense(128, activation="relu"),
          tf.keras.layers.Dense(64, activation="relu"),
          tf.keras.layers.Dense(1),
        ])
    
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
          metrics = tfrs.metrics.FactorizedTopK(
            candidates = product.batch(128).map(self.product_model)
        )
    )

    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
          loss = tf.keras.losses.MeanSquaredError(),
          metrics = [tf.keras.metrics.RootMeanSquaredError()]
    )

    self.retrieval_weight = retrieval_weight
    self.rating_weight = rating_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    user_embeddings = self.user_model(features["User_Id"])
    product_embeddings = self.product_model(features["Class_Id"])
    return(
        user_embeddings, product_embeddings, self.rating_model(tf.concat([user_embeddings, product_embeddings], axis = 1),)
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training = False) -> tf.Tensor:
    ratings = features.pop("Order_Rating")
    user_embeddings, product_embeddings, rating_predictions = self(features)

    retrieval_loss = self.retrieval_task(user_embeddings, product_embeddings)
    rating_loss = self.rating_task(
        labels = ratings,
        predictions = rating_predictions
    )
    return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)

In [101]:
model = Model(rating_weight=1, retrieval_weight=1)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [102]:
model.fit(cached_train, epochs=50)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"top 5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.3f}.")
print(f"RMSE Ranking: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/50
7/7 [==============================] - 2s 62ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0650 - factorized_top_k/top_10_categorical_accuracy: 0.1675 - factorized_top_k/top_50_categorical_accuracy: 0.9862 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - root_mean_squared_error: 2.3928 - loss: 498.4463 - regularization_loss: 0.0000e+00 - total_loss: 498.4463
Epoch 2/50
7/7 [==============================] - 0s 63ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.4250 - factorized_top_k/top_10_categorical_accuracy: 0.7312 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - root_mean_squared_error: 1.3953 - loss: 492.5746 - regularization_loss: 0.0000e+00 - total_loss: 492.5746
Epoch 3/50
7/7 [==============================] - 0s 49ms/step - factorized_top_k/top_1_categorical_ac

1/1 [==============================] - 1s 551ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0061 - factorized_top_k/top_5_categorical_accuracy: 0.0610 - factorized_top_k/top_10_categorical_accuracy: 0.1890 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - root_mean_squared_error: 1.6796 - loss: 1129.4384 - regularization_loss: 0.0000e+00 - total_loss: 1129.4384
top 5 accuracy: 0.061.
RMSE Ranking: 1.680.


In [103]:
retrieval = tfrs.layers.factorized_top_k.ScaNN(model.user_model, k= 3, num_leaves = 10)
retrieval.index_from_dataset(
    tf.data.Dataset.zip((product.batch(5), product.batch(5).map(model.product_model))))
_ = retrieval(np.array(["29"]))
tf.saved_model.save(retrieval, "/content/sample_data/Model", options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"]))

INFO:tensorflow:Assets written to: /content/sample_data/Model/assets


INFO:tensorflow:Assets written to: /content/sample_data/Model/assets


In [104]:
load = tf.saved_model.load("/content/sample_data/Model")

In [111]:
rating, suggestion = load(np.array(["110"]))
print(rating)
print(suggestion)

tf.Tensor([[8.050923  3.828683  3.6267118]], shape=(1, 3), dtype=float32)
tf.Tensor([[b'27' b'39' b'32']], shape=(1, 3), dtype=string)
